# API para relato no Fiscaliza da Anatel

In [1]:
import os
import random
from random import randint
from dotenv import load_dotenv
from fiscaliza.main import Fiscaliza
from fiscaliza.constants import MUNICIPIOS, SERVICOS
from fiscaliza.attrs import FIELDS
load_dotenv(override=True)

True

## Autenticação e Conexão com o servidor do Redmine
A classe `Fiscaliza` é somente para efetuar a conexão e autenticação com o servidor do Redmine. 

In [7]:
issue_id = "124182"
fiscaliza = Fiscaliza(os.environ["USERNAME"], os.environ["PASSWORD"], teste=True)
issue = fiscaliza.get_issue(issue_id)

## Conexão com dada inspeção 
O método `get_issue` instancia e retorna um objeto do tipo `Issue`, este que encapsula a funcionalidade principal de resgate de informações, validação e formatação de informações e atualização de inspeção.
O mesmo objeto pode ser instanciado diretamente passando o `client` do objeto `Fiscaliza` após autenticado e o `id` da inspeção.
```
issue = Issue(fiscaliza.client, issue_id)
```

## Informações da Inspeção
Todas as informações principais da Inspeção estão disponíveis através do atributo `attrs`

In [8]:
# |filter_stream membros
issue.attrs

{'acao': {'type': 'acao_de_inspecao',
  'status': 'Em andamento',
  'name': 'ACAO_GR08_2024_0014',
  'description': '[PMEC 2024 Etapa 2] Monitorar canais e faixas de frequências relacionados às aplicações críticas (como, por exemplo, radionavegação e radiocomunicação aeronáutica e canais de emergência) na forma a ser estabelecida no Plano de Ação de Fiscalização.\r\n'},
 'agrupamento': '',
 'allowed_statuses': '',
 'anexos': ['https://sistemasnethm/fiscaliza/attachments/download/9190/Info.json',
  'https://sistemasnethm/fiscaliza/attachments/download/9191/Info.json'],
 'ano_de_execucao': 2024,
 'app_fiscaliza': 0,
 'assigned_to': 'Eric Magalhães Delgado',
 'atualizacao': 'Atualizada por Ronaldo da Silva Alves Batista em 25/07/2024 às 05:05:36',
 'author': 'Eric Magalhães Delgado',
 'changesets': '',
 'children': '',
 'classe_da_inspecao': 'Técnica',
 'closed_on': '',
 'coordenacao_responsavel': 'FI2',
 'created_on': '2024-04-18T12:38:03Z',
 'data_de_inicio_efetivo': '2024-07-25',
 'des

O atributo acima retorna um dicionário com atributos atuais da Inspeção no Fiscaliza. Existem campos editáveis através da API ou da interface Web e outros meramente informativos, além de alguns campos presentes no `Redmine` mas sem uso na instância Fiscaliza da Anatel. Um exemplo desses é o atributo `children`, sempre vazio.

## Campos editáveis
Os dados que são editáveis, tanto pela API quanto pela interface web aparecem em `minúsculo` no dicionário. Para estes dados temos um outro método mais informativo:

In [4]:
issue.editable_fields

{'status': (value: Rascunho),
 'description': (value: [PMEC 2024 Etapa 2] Monitorar canais e faixas de frequências relacionados às aplicações críticas (como, por exemplo, radionavegação e radiocomunicação aeronáutica e canais de emergência) na forma a ser estabelecida no Plano de Ação de Fiscalização.
 ) | <mandatory>,
 'start_date': (value: 2024-03-01) | <mandatory>,
 'due_date': (value: 2024-11-30) | <mandatory>,
 'tipo_de_inspecao': (value: ) | <mandatory>, <options>, conditional,
 'fiscal_responsavel': (value: ) | <mandatory>, <options>,
 'fiscais': (value: []) | <mandatory>, <multiple>, <options>,
 'entidade_da_inspecao': (value: []), <multiple>,
 'ufmunicipio': (value: []) | <mandatory>, <multiple>, <options>,
 'servicos_da_inspecao': (value: []) | <mandatory>, <multiple>, <options>,
 'procedimentos': (value: []) | <mandatory>, <multiple>, <options>, conditional,
 'horas_de_preparacao': (value: 0) | <mandatory>,
 'horas_de_deslocamento': (value: 0) | <mandatory>,
 'horas_de_execu

Nesse caso as chaves são os nomes dos campos e os valores são os objetos que modelam o campo. Ao mostrar ou imprimir o dicionário, como mostrado acima, é impresso o valor atual do campo entre parênteses.
```
'tipo_de_inspecao': (value: Uso do Espectro - Não Outorgado) | <mandatory>, <options>, conditional
```
Separado pela barra vertical são mostrados alguns atributos do campo:

  - `<mandatory>` Atributo booleano indicando se o campo é obrigatório;
  - `<multiple>` Atributo booleano. `False`: Campo de valor único. `True` Campo aceita mais de um valor e por conseguinte deve ser fornecido uma lista em vez de um valor atômico ( `int`, `float`, `str` );
  - `<options>` Atributo do tipo lista. Quando presente sinaliza que o campo é restrito e só aceita  valores pré-definidos;
  - `conditional` Esse não é um atributo, somente um sinalizador indicando que o campo é condicional e para dado valor preenchido neste campo, ficam disponíveis outros campos, estes podem ser obrigatórios e inclusive condicionais.

O valor em si do campo é guardado no atributo `value`, acessado através da notação de ponto 

In [9]:
issue.editable_fields['tipo_de_inspecao'].value

'Outorga - Aspectos não Técnicos'

### Campos Obrigatórios e Condicionais

Os dois métodos a seguir filtram, dentre os campos editáveis, os campos obrigatório e condicionais:

In [10]:
issue.mandatory_fields()

{'description': (value: [PMEC 2024 Etapa 2] Monitorar canais e faixas de frequências relacionados às aplicações críticas (como, por exemplo, radionavegação e radiocomunicação aeronáutica e canais de emergência) na forma a ser estabelecida no Plano de Ação de Fiscalização.
 ) | <mandatory>,
 'start_date': (value: 2024-03-01) | <mandatory>,
 'due_date': (value: 2024-05-30) | <mandatory>,
 'tipo_de_inspecao': (value: Outorga - Aspectos não Técnicos) | <mandatory>, <options>, conditional,
 'fiscal_responsavel': (value: Eric Magalhães Delgado) | <mandatory>, <options>,
 'fiscais': (value: ['Eric Magalhães Delgado', 'Ronaldo da Silva Alves Batista']) | <mandatory>, <multiple>, <options>,
 'ufmunicipio': (value: ['MG/Bocaina de Minas', 'AL/União dos Palmares']) | <mandatory>, <multiple>, <options>,
 'servicos_da_inspecao': (value: ['099 - RESTRITO - SERVIÇO DE RADIAÇÃO RESTRITA', '032 - COLETIVO - RADIOENLACES ASSOCIADOS AO SERVIÇO REDE TRANSPORTE TELECOMUNICAÇÃO']) | <mandatory>, <multiple>,

In [11]:
issue.conditional_fields()

{'tipo_de_inspecao': (value: Outorga - Aspectos não Técnicos) | <mandatory>, <options>, conditional,
 'procedimentos': (value: ['Análise/coleta de Dados']) | <mandatory>, <multiple>, <options>, conditional,
 'pai_instaurado_pela_anatel': (value: Sim) | <mandatory>, <options>, conditional,
 'precisa_reservar_instrumentos': (value: 0) | <mandatory>, <options>, conditional,
 'utilizou_algum_instrumento': (value: 0) | <mandatory>, <options>, conditional}

> O conjunto de valores para os campos com opções pré-definidas estão relacionados no atributo `.options`

In [12]:
issue.editable_fields['tipo_de_inspecao'].options

['',
 'Bloqueio Administrativo',
 'Certificação',
 'Medição de CEMRF (RNI)',
 'Outorga - Aspectos não Técnicos',
 'Outorga - Aspectos Técnicos',
 'Uso do Espectro - Interferência',
 'Uso do Espectro - Monitoração',
 'Uso do Espectro - Não Outorgado']

In [13]:
issue.editable_fields['procedimentos'].options

['Nenhum',
 'Lacração',
 'Apreensão',
 'Interrupção',
 'Não Cadastrado',
 'Notificado',
 'A Notificar',
 'Liberação/Desinterrupção',
 'Orientação ao Usuário',
 'Comunicado',
 'Deslacrado',
 'Vistoriado',
 'Emissão Termo Violação de Lacre',
 'Apoio a busca e apreensão',
 'Investigação/Pesquisa',
 'Não Lacrado - Impedimento',
 'Não Lacrado - Amparo Judicial',
 'Não Lacrado - Responsável Ausente',
 'Não Lacrado - Local Fechado',
 'Constatação Violação Lacre/Relacrado',
 'Constatação Violação Lacre/Impedimento',
 'Notícia Crime',
 'Monitorado alterado',
 'Constatação Encerramento - Informe',
 'Levantamento de Dados',
 'Análise/coleta de Dados',
 'Monitorado',
 'Não Lacrado - Desativado',
 'Devolução de Produto(s)',
 'Outros']

## Atualizando os campos editáveis da Inspeção
Dada a existência de campos condicionais, para obtermos todos os campos editáveis para dado tipo de inspeção, é preciso fornecer um dicionário de dados para que os campos sejam atualizados de acordo com os valores fornecidos para os campos condicionais.

In [14]:
dados = {
    "tipo_de_inspecao": "Certificação",
    "precisa_reservar_instrumentos": "0",
    "utilizou_algum_instrumento": "0",
    "procedimentos": "Lacração",
}

A atualização dos campos é feita passando o dicionário para o método `update_fields`.

In [15]:
issue.update_fields(dados)

Depois de atualizado os campos com o método acima, checamos novamente o atributo `editable_fields`

In [16]:
issue.editable_fields

{'status': (value: Relatando),
 'description': (value: [PMEC 2024 Etapa 2] Monitorar canais e faixas de frequências relacionados às aplicações críticas (como, por exemplo, radionavegação e radiocomunicação aeronáutica e canais de emergência) na forma a ser estabelecida no Plano de Ação de Fiscalização.
 ) | <mandatory>,
 'start_date': (value: 2024-03-01) | <mandatory>,
 'due_date': (value: 2024-05-30) | <mandatory>,
 'tipo_de_inspecao': (value: Certificação) | <mandatory>, <options>, conditional,
 'fiscal_responsavel': (value: Eric Magalhães Delgado) | <mandatory>, <options>,
 'fiscais': (value: ['Eric Magalhães Delgado', 'Ronaldo da Silva Alves Batista']) | <mandatory>, <multiple>, <options>,
 'entidade_da_inspecao': (value: []), <multiple>,
 'ufmunicipio': (value: ['MG/Bocaina de Minas', 'AL/União dos Palmares']) | <mandatory>, <multiple>, <options>,
 'servicos_da_inspecao': (value: ['099 - RESTRITO - SERVIÇO DE RADIAÇÃO RESTRITA', '032 - COLETIVO - RADIOENLACES ASSOCIADOS AO SERVIÇO

Caso o conjunto de dados já seja conhecido para dado tipo de inspeção, basta passar o dicionário diretamente para o método `update`. Internamente os métodos mostrados anteriormente de atualização dos campos são chamados.

## Exemplo completo
O dicionário a seguir contém valores misturados de diversos tipos de inspeção distintos, além de valores aleatorizados. A API simplesmente filtra os valores que se aplicam para o tipo de inspeção solicitada e o valor do dado fornecido para os campos condicionais. As demais chaves que não se aplicam são ignoradas.

In [17]:
# | code-fold: true
dados = {
    "endereco_da_inspecao": "Rua Machado de Assis, 27 - Morro Grande, Rio de Janeiro - RJ",
    "campo_eletrico__pico_vm": randint(50, 100),
    "campo_eletrico_rms_vm": randint(20, 100),
    "coordenacao_responsavel": "FI2",
    "cnpjcpf_da_entidade": "27865757000102",
    "entidade_com_cadastro_stel": "Não",
    "entidade_outorgada": randint(0, 1),
    "esta_em_operacao": randint(0, 1),
    "numero_da_estacao": "1493671",
    "fiscais": ["Eric Magalhães Delgado", "Ronaldo da Silva Alves Batista"],
    "fiscal_responsavel": "Eric Magalhães Delgado",
    "foi_constatada_interferencia": randint(0, 1),
    "frequencia_inicial": randint(70, 110),
    "frequencia_final": randint(110, 117),
    "gerar_relatorio": randint(0, 1),
    "gerar_plai": randint(0, 1),
    "tipo_do_processo_plai": random.choice(FIELDS["tipo_do_processo_plai"].options),
    "coord_fi_plai": random.choice(FIELDS["coord_fi_plai"].options),
    "html_path": "/mnt/c/Users/rsilva/code/fiscaliza/tests/Report_2024.02.18_T11.30.55_123456.html",
    "uploads": [
        {
            "path": "/mnt/c/Users/code/fiscaliza/tests/Report_2024.02.18_T11.30.55_123456.json",
            "filename": "Info.json",
        }
    ],
    "identificada_a_origem": str(randint(0, 1)),
    "unidade_da_frequencia_final": "MHz",
    "unidade_da_frequencia_inicial": "MHz",
    "horas_de_conclusao": randint(1, 8),
    "horas_de_deslocamento": randint(1, 8),
    "horas_de_execucao": randint(8, 40),
    "horas_de_preparacao": randint(1, 8),
    "houve_obice": random.randint(0, 1),
    "houve_interferencia": random.choice(["Sim", "Não"]),
    "latitude_coordenadas": -randint(0, 33),
    "longitude_coordenadas": -randint(34, 73),
    "latitude_da_estacao": -randint(0, 33),
    "longitude_da_estacao": -randint(34, 73),
    "nome_da_entidade": "Globo S/A",
    "numero_do_pai": "123456",
    "observacao_tecnica_amostral": "Simulação com o HTZ",
    "pai_instaurado_pela_anatel": random.choice(["Sim", "Não"]),
    "precisa_reservar_instrumentos": "0",  #
    "procedimentos": random.choices(FIELDS["procedimentos"].options, k=2),  #
    "qnt_produt_lacradosapreend": "0",
    "reserva_de_instrumentos": "0",
    "no_sav": "Teste de Quebra\n de linha",
    "documento_instaurador_do_pado": "0201235\n0201239\n0201237",
    "no_do_lacre": "50",
    "motivo_de_lai": random.choice(FIELDS["motivo_de_lai"].options),
    "no_sei_do_aviso_lai": "",
    "sanada_ou_mitigada": random.choice(["0", "1"]),
    "servicos_da_inspecao": random.choices(list(SERVICOS.values()), k=2),
    "situacao_constatada": "Irregular",
    "situacao_de_risco_a_vida": "Sim",
    "tipo_de_inspecao": random.choice(FIELDS["tipo_de_inspecao"].options),
    "ufmunicipio": random.choices(MUNICIPIOS, k=2),
    "uso_de_produto_homologado": random.choice(["0", "1"]),
    "utilizou_algum_instrumento": "0",
    "utilizou_apoio_policial": random.choice(FIELDS["utilizou_apoio_policial"].options),
    "utilizou_tecnicas_amostrais": random.choice(
        FIELDS["utilizou_tecnicas_amostrais"].options
    ),
    "description": "[PMEC 2024 Etapa 2] Monitorar canais e faixas de frequências relacionados às aplicações críticas (como, por exemplo, radionavegação e radiocomunicação aeronáutica e canais de emergência) na forma a ser estabelecida no Plano de Ação de Fiscalização.\r\n",
    "start_date": "2024-03-01",
    "due_date": "2024-05-30",
}

In [18]:
issue.update(dados)

ValueError: Arquivo C:\Users\rsilva\code\fiscaliza\tests\Report_2024.02.18_T11.30.55_123456.json não existe ou não é um arquivo

## Gerar Relatório de Atividades
Para as inspeções de `Certificação` e `Uso do Espectro - Monitoração` é possível gerar o relatório de atividades no SEI por meio do Fiscaliza, para tal basta fornecer o valor: 
```
dados['gerar_relatório'] = '1'
```


In [ ]:
dados['gerar_relatorio'] = '1'
issue.update(dados)

Atualizado para o status Relatando


In [ ]:
issue.attrs

{'acao': {'type': 'acao_de_inspecao',
  'status': 'Em andamento',
  'name': 'ACAO_GR08_2024_0014',
  'description': '[PMEC 2024 Etapa 2] Monitorar canais e faixas de frequências relacionados às aplicações críticas (como, por exemplo, radionavegação e radiocomunicação aeronáutica e canais de emergência) na forma a ser estabelecida no Plano de Ação de Fiscalização.\r\n'},
 'agrupamento': '',
 'allowed_statuses': '',
 'anexos': ['https://sistemasnethm/fiscaliza/attachments/download/9190/Info.json'],
 'ano_de_execucao': 2024,
 'app_fiscaliza': 0,
 'assigned_to': 'Eric Magalhães Delgado',
 'atualizacao': 'Atualizada por Ronaldo da Silva Alves Batista em 25/07/2024 às 05:03:48',
 'author': 'Eric Magalhães Delgado',
 'changesets': '',
 'children': '',
 'classe_da_inspecao': 'Técnica',
 'closed_on': '',
 'coordenacao_responsavel': 'FI2',
 'created_on': '2024-04-18T12:38:03Z',
 'data_de_inicio_efetivo': '2024-07-25',
 'description': '[PMEC 2024 Etapa 2] Monitorar canais e faixas de frequências 

> Para substituir o relatório de atividades, basta refazer a chamada fornecendo o valor `1` para a chave `gerar_relatorio`.

In [ ]:
dados["gerar_relatorio"] = "1"
issue.update(dados)

Atualizado para o status Relatando


In [ ]:
issue.attrs

{'acao': {'type': 'acao_de_inspecao',
  'status': 'Em andamento',
  'name': 'ACAO_GR08_2024_0014',
  'description': '[PMEC 2024 Etapa 2] Monitorar canais e faixas de frequências relacionados às aplicações críticas (como, por exemplo, radionavegação e radiocomunicação aeronáutica e canais de emergência) na forma a ser estabelecida no Plano de Ação de Fiscalização.\r\n'},
 'agrupamento': '',
 'allowed_statuses': '',
 'anexos': ['https://sistemasnethm/fiscaliza/attachments/download/9190/Info.json'],
 'ano_de_execucao': 2024,
 'app_fiscaliza': 0,
 'assigned_to': 'Eric Magalhães Delgado',
 'atualizacao': 'Atualizada por Ronaldo da Silva Alves Batista em 25/07/2024 às 05:03:48',
 'author': 'Eric Magalhães Delgado',
 'changesets': '',
 'children': '',
 'classe_da_inspecao': 'Técnica',
 'closed_on': '',
 'coordenacao_responsavel': 'FI2',
 'created_on': '2024-04-18T12:38:03Z',
 'data_de_inicio_efetivo': '2024-07-25',
 'description': '[PMEC 2024 Etapa 2] Monitorar canais e faixas de frequências 

> Caso o intuito seja somente editar o campo `Relatório de Atividades`, basta fornecer o valor para a chave `relatorio_de_atividades` e não esquecer de fornecer o valor `0` para a chave `gerar_relatório` ou simplesmente excluí-la.  

In [ ]:
dados['relatorio_de_atividades'] = '0206885'
dados['gerar_relatorio'] = '0'
issue.update(dados)

Atualizado para o status Relatando
